In [ ]:
# insource the data from website with the dat file

ds = pd.read_csv('http://www.stat.ufl.edu/~winner/data/airq402.dat',delim_whitespace=1,header = None)

In [ ]:
#drop the field

df.drop(["ID","Personal Loan"],axis=1)
books.drop(["imageUrlS","imageUrlM","imageUrlL"],axis = 1, inplace = True)

In [ ]:
# display data frame with some group by
df.groupby(['status']).size()

In [ ]:
#list the details of nul and unique count and details 

listItem = []
for col in df.columns :
    listItem.append([col,df[col].dtype,
                     df[col].isnull().sum(),
                     round((df[col].isnull().sum()/len(df[col])) * 100,2),
                    df[col].nunique(),
                     list(df[col].sample(5).drop_duplicates().values)]);

dfDesc = pd.DataFrame(columns=['dataFeatures', 'dataType', 'null', 'nullPct', 'unique', 'uniqueSample'],
                     data=listItem)
dfDesc

In [ ]:
# conditional display with the few columns in the data frame 

pd.DataFrame(df[df["Experience"]<0][["Age","Experience"]].sort_values("Age"))

In [ ]:
# display the fields with condition 
books[books['yearOfPublication'] == 'DK Publishing Inc']

In [ ]:
# drop the fileds that are not required
books = (books.drop([209538,221678,220731]))

In [ ]:
#Change the data type
mpg_df['hp'] = mpg_df['hp'].astype(float)

In [ ]:
#Get all unique values in ascending order for column Age
users[["Age"]].sort_values("Age")["Age"].unique()
print(sorted(users.Age.unique()))

In [ ]:
#replace numpy nan with median. 
mpg_df['hp'].replace(np.nan, mpg_df['hp'].median(), inplace = True)

In [ ]:
ratings[ratings.ISBN.isin(books.ISBN)]

In [ ]:
Outlier imputation function with median

 def Impute_Outliers(df,method = "median"):
        
        df_Columns = df.columns
        Subset_Columns = pd.Series(df.select_dtypes(include=['int32','int64','float64','float32']).columns)

        Subset = df[Subset_Columns]
        
        IQR = Subset.quantile(0.75) - Subset.quantile(0.25)
        
        Q3_values = Subset.quantile(0.75) + (1.5 * IQR)
        Q1_values = Subset.quantile(0.25) - (1.5 * IQR)

        Q1 = []
        for i in range(1,len(Subset_Columns)+1):
            c = "Q1"+str(i)
            Q1.append(c)

        Q3 = []
        for i in range(1,len(Subset_Columns)+1):
            c = "Q3"+str(i)
            Q3.append(c)

        df[Q3] = Subset > Q3_values[0:len(Subset_Columns)]
        df[Q1] = Subset < Q1_values[0:len(Subset_Columns)]

        Q1_Outliers = []
        Q1_j = []
        Q3_Outliers = []
        Q3_j = []
        for i in range(0,len(Q1)):
            i = Q1[i]
            No = df.shape[0] - df[i].value_counts()[0]
            Q1_Outliers.append(No)
            Q1_j.append(i)
        Q1_Col = pd.DataFrame(Q1_j, columns=["Q1"])
        Q1_outliers = pd.DataFrame(Q1_Outliers, columns=["Q1 Outliers"])
        Outliers_impact_Q1 = Q1_Col.join(Q1_outliers)

        for i in range(0,len(Q3)):
            i = Q3[i]
            No = df.shape[0] - df[i].value_counts()[0]
            Q3_Outliers.append(No)
            Q3_j.append(i)
        Q3_Col = pd.DataFrame(Q3_j, columns=["Q3"])
        Q3_outliers = pd.DataFrame(Q3_Outliers, columns=["Q3 Outliers"])
        Outliers_impact_Q3 = Q3_Col.join(Q3_outliers)

        Outliers_impact = Outliers_impact_Q1['Q1 Outliers']+Outliers_impact_Q3['Q3 Outliers']
        Outliers_impact = (pd.DataFrame(Subset_Columns, columns=["Column Name"])).join(pd.DataFrame(Outliers_impact, columns=["No of Outliers"]))


        aij = []
        for i in range(0,len(Q3)):
            i = Q3[i]
            bij = ((pd.DataFrame(df[i])).index[(df[i] == True)].tolist())
            aij = aij + bij
        Q3_indices = (pd.Series(aij)).value_counts()


        cij = []
        for i in range(0,len(Q1)):
            i = Q1[i]
            dij = ((pd.DataFrame(df[i])).index[(df[i] == True)].tolist())
            cij = cij + dij
        Q1_indices = (pd.Series(cij)).value_counts()

        print("No of records impacted by Outliers = ", Outliers_impact['No of Outliers'].sum())
        print("No of records in outliers beyond Q4 = ", (pd.DataFrame(Q3_Outliers)[0]).sum())
        print("No of records in outliers beyond Q1 = ", (pd.DataFrame(Q1_Outliers)[0]).sum())
        print("No of records imputed using the",method,"is", Outliers_impact['No of Outliers'].sum())

        Outliers_Q3_Q1 = pd.DataFrame(Q3_values, columns = ['Q3_values']).join(pd.DataFrame(Q1_values, columns=['Q1_values']))
        for i in range(0,len(Subset_Columns)):
            Q3 = ((Outliers_Q3_Q1).T)[Subset_Columns[i]].loc['Q3_values']
            Q1 = ((Outliers_Q3_Q1).T)[Subset_Columns[i]].loc['Q1_values']
            df.loc[df[Subset_Columns[i]] > Q3, Subset_Columns[i]] = numpy.nan
            df.loc[df[Subset_Columns[i]] < Q1, Subset_Columns[i]] = numpy.nan
            if method == "median":
                median1 = ((df.loc[(df[Subset_Columns[i]]<((((Outliers_Q3_Q1).T)[Subset_Columns[i]])['Q3_values'])) & 
                     (df[Subset_Columns[i]]>((((Outliers_Q3_Q1).T)[Subset_Columns[i]])['Q1_values']))])[Subset_Columns[i]]).median()
            else:
                median1 = ((df.loc[(df[Subset_Columns[i]]<((((Outliers_Q3_Q1).T)[Subset_Columns[i]])['Q3_values'])) & 
                     (df[Subset_Columns[i]]>((((Outliers_Q3_Q1).T)[Subset_Columns[i]])['Q1_values']))])[Subset_Columns[i]]).mean()
            df.replace(numpy.nan,median1,inplace= True)
        df = df.iloc[:,0:len(df_Columns)]
        return df


In [ ]:
# box and whisker plot
df.plot(kind='box', subplots=True, layout=(4,5), fontsize=8, figsize=(14,14))
pyplot.show()

In [ ]:
# histogram for the data frame
df.hist(figsize=(10,15),bins = 50,xlabelsize=8,ylabelsize =8,)
#plt.show()

In [ ]:

#histogram from one of th field
df['yr_built'].hist(figsize=(10,15),bins = 50,xlabelsize=8,ylabelsize =8,)

In [ ]:

# seperate only the continious data into seperate data frame
ContiniousDataColumn = pd.Series(df.select_dtypes(include=['int32','int64','float64','float32']).columns)
ContinousData = df[ContiniousDataColumn]

In [ ]:
# TRansformations and imputations 

In [ ]:
dataset = pd.read_csv('Data.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 3].values


In [ ]:
from sklearn.preprocessing import Imputer
imputer = Imputer(missing_values = 'NaN',  strategy = 'mean', axis = 0)
imputer = imputer.fit(X[:, 1:3])
X[:, 1:3] = imputer.transform(X[:, 1:3])


In [ ]:
# One hot encoder

from sklearn.preprocessing import  LabelEncoder, OneHotEncoder
labelencoder_X = LabelEncoder()
X[:, 0] = labelencoder_X.fit_transform(X[:, 0])
onehotencoder =  OneHotEncoder(categorical_features = [0])
X = onehotencoder.fit_transform(X).toarray()


In [ ]:
labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y)


In [ ]:
from sklearn.cross_validation import  train_test_split
X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size = 0.2,  random_state = 42)

In [ ]:
from sklearn.preprocessing import StandardScaler  
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)  
X_test = sc_X.transform(X_test)

In [ ]:
# LAMBDA function

In [1]:
f = lambda x, y : x + y
f(1,1)

2

In [2]:
z = lambda i,j,k : i*j*k
z(1,2,3)


6

#### The map() Function

map() is a function with two arguments:

r = map(func, seq)

The first argument func is the name of a function and the second a sequence (e.g. a list) seq. map() applies the function func to all the elements of the sequence seq. It returns a new list with the elements changed by func

In [ ]:
def fahrenheit(T):
    return ((float(9)/5)*T + 32)

def celsius(T):
    return (float(5)/9)*(T-32)

In [ ]:
temp = (36.5, 37, 37.5,39)

F = map(fahrenheit, temp)

print F

C = map(celsius, F)

print C

By using lambda, we wouldn't have had to define and name the functions fahrenheit() and celsius().

In [ ]:
Celsius = [39.2, 36.5, 37.3, 37.8]

Fahrenheit = map(lambda x: (float(9)/5)*x + 32, Celsius)

print Fahrenheit

C = map(lambda x: (float(5)/9)*(x-32), Fahrenheit)

print C